수원시 유입의 이유를 알기 위함이므로 거주 10년 이상인 경우는 제외하고 확인

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving (공개용)2025년 2분기 패널조사_데이터(가중치).xlsx to (공개용)2025년 2분기 패널조사_데이터(가중치).xlsx


In [ ]:
import pandas as pd

df = pd.read_excel('(공개용)2025년 2분기 패널조사_데이터(가중치).xlsx')  # 실제 파일명으로 바꿔줘
filtered_df = df[df['q9'] != 5]

# filtered_df = df[~((df['DM2_1'] == 1) & (df['q9'] == 5))]
df.head()
print(f"전체 응답자 수: {len(df)}")
print(f"필터링된 응답자 수: {len(filtered_df)}")

전체 응답자 수: 1587
필터링된 응답자 수: 580


q2 관심도만 확인

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 예시 데이터프레임: df
# q2: 1~7점 척도 (1=전혀 관심 없음, 7=매우 관심 많음)

# 1. q2 관심도 분포표 (빈도 + 비율)
q2_counts = df['q2'].value_counts().sort_index()
q2_percent = round((q2_counts / len(df)) * 100, 2)

q2_dist = pd.DataFrame({
    '관심도 점수': q2_counts.index,
    '응답자 수': q2_counts.values,
    '백분율(%)': q2_percent.values
})

print(q2_dist)



   관심도 점수  응답자 수  백분율(%)
0       1      5    0.32
1       2     21    1.32
2       3     78    4.91
3       4    308   19.41
4       5    517   32.58
5       6    411   25.90
6       7    247   15.56


q3 만족도만 확인

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. q3 만족도 분포표 계산
q3_counts = df['q3'].value_counts().sort_index()
q3_percent = round((q3_counts / len(df)) * 100, 1)

q3_dist = pd.DataFrame({
    '만족도 점수': q3_counts.index,
    '응답자 수': q3_counts.values,
    '백분율(%)': q3_percent.values
})

# 2. 표 출력
print(q3_dist.to_string(index=False))

 만족도 점수  응답자 수  백분율(%)
      1      3     0.2
      2     25     1.6
      3     72     4.5
      4    296    18.7
      5    538    33.9
      6    451    28.4
      7    202    12.7


연령별 관심도, 만족도 분포

In [ ]:
import pandas as pd

# 연령별 q2 분포 계산 (비율)
q2_by_age = (
    filtered_df
    .groupby('DM2_1')['q2']
    .value_counts(normalize=True)  # 각 연령 그룹 내에서 퍼센트
    .mul(100)
    .round(1)
    .unstack(fill_value=0)  # 행: 연령, 열: 관심도 점수(1~7)
)
q3_by_age = (
    filtered_df
    .groupby('DM2_1')['q3']
    .value_counts(normalize=True)  # 각 연령 그룹 내에서 퍼센트
    .mul(100)
    .round(1)
    .unstack(fill_value=0)  # 행: 연령, 열: 관심도 점수(1~7)
)
# 결과 출력
print("※ 연령별 정책 관심도(q2) 분포 [%]")
print(q2_by_age)
print("※ 연령별 정책 만족도(q3) 분포 [%]")
print(q3_by_age)

※ 연령별 정책 관심도(q2) 분포 [%]
q2       2     3     4     5     6    7
DM2_1                                  
1      3.6  14.3  26.2  33.3  19.0  3.6
2      0.7   4.5  29.4  33.8  26.4  5.2
3      0.0   3.2   8.6  48.9  31.7  7.5
4      0.0   0.0  19.0  47.6  28.6  4.8
5      5.0   5.0  25.0  30.0  30.0  5.0
※ 연령별 정책 만족도(q3) 분포 [%]
q3       2     3     4     5     6     7
DM2_1                                   
1      0.0  10.7  19.0  40.5  26.2   3.6
2      0.7   2.6  16.4  42.8  33.8   3.7
3      1.1   2.7  11.3  49.5  29.0   6.5
4      0.0   0.0  23.8  28.6  38.1   9.5
5      0.0  10.0  30.0  25.0  25.0  10.0


연령별 만족, 불만족 이유 분포 확인

In [ ]:
# 1. 만족자만 필터링 (q3 >= 5)
satisfied = filtered_df[filtered_df['q3'] >= 5].copy()

# 2. 이유 1순위와 2순위 결합 → 하나의 시리즈로 묶기
satisfied_melted = pd.concat([
    satisfied[['DM2_1', 'q3_1_1']].rename(columns={'q3_1_1': '이유'}),
    satisfied[['DM2_1', 'q3_1_2']].rename(columns={'q3_1_2': '이유'})
])

# 3. 연령별 만족 이유 분포 (%) 계산
satisfied_reason_dist = (
    satisfied_melted
    .groupby('DM2_1')['이유']
    .value_counts(normalize=True)
    .mul(100)
    .round(1)
    .unstack(fill_value=0)
)

print("✅ 연령별 정책 만족 이유 분포 [%]")
print(satisfied_reason_dist)


✅ 연령별 정책 만족 이유 분포 [%]
이유      1.0   2.0   3.0   4.0   5.0   6.0  7.0  8.0
DM2_1                                              
1       7.8  23.3  17.2  17.2  14.7  12.9  5.2  1.7
2      13.2  30.2  21.2  13.7   8.0  11.3  2.4  0.0
3       4.1  24.1  20.3  14.5  29.3   4.1  3.4  0.0
4      28.1  21.9  21.9   9.4   9.4   6.2  3.1  0.0
5      20.8  29.2  20.8  12.5   4.2   8.3  4.2  0.0


In [ ]:
# 1. 불만족자만 필터링 (q3 <= 3)
unsatisfied = filtered_df[filtered_df['q3'] <= 3].copy()

# 2. 이유 1순위와 2순위 결합
unsatisfied_melted = pd.concat([
    unsatisfied[['DM2_1', 'q3_2_1']].rename(columns={'q3_2_1': '이유'}),
    unsatisfied[['DM2_1', 'q3_2_2']].rename(columns={'q3_2_2': '이유'})
])

# 3. 연령별 불만족 이유 분포 (%) 계산
unsatisfied_reason_dist = (
    unsatisfied_melted
    .groupby('DM2_1')['이유']
    .value_counts(normalize=True)
    .mul(100)
    .round(1)
    .unstack(fill_value=0)
)

print("✅ 연령별 정책 불만족 이유 분포 [%]")
print(unsatisfied_reason_dist)


✅ 연령별 정책 불만족 이유 분포 [%]
이유      1.0   2.0   3.0   4.0   5.0   6.0  7.0   8.0
DM2_1                                               
1       0.0  11.1  33.3   5.6  38.9   5.6  5.6   0.0
2       5.6  11.1  33.3  33.3   5.6  11.1  0.0   0.0
3      25.0  16.7  25.0  16.7   0.0   0.0  8.3   8.3
5      25.0  25.0   0.0  25.0   0.0   0.0  0.0  25.0


주거구별 관심도, 만족도 분

In [ ]:
import pandas as pd

# 주거구별 q2 분포 계산 (비율)
q2_by_gu = (
    filtered_df
    .groupby('GU')['q2']
    .value_counts(normalize=True)  # 각 GU 그룹 내에서 퍼센트
    .mul(100)
    .round(1)
    .unstack(fill_value=0)  # 행: GU, 열: 관심도 점수(1~7)
)
gu_names = {1: '장안구', 2: '권선구', 3: '팔달구', 4: '영통구'}
q2_by_gu.index = q2_by_gu.index.map(gu_names)
# 결과 출력
print("※ 주거구별 정책 관심도(q2) 분포 [%]")
print(q2_by_gu)
# 주거구별 q3 분포 계산 (비율)
q3_by_gu = (
    filtered_df
    .groupby('GU')['q3']
    .value_counts(normalize=True)  # 각 GU 그룹 내에서 비율
    .mul(100)
    .round(1)
    .unstack(fill_value=0)  # 행: GU, 열: 만족도 점수(1~7)
)
gu_names = {1: '장안구', 2: '권선구', 3: '팔달구', 4: '영통구'}
q3_by_gu.index = q3_by_gu.index.map(gu_names)
# 결과 출력
print("※ 주거구별 정책 만족도(q3) 분포 [%]")
print(q3_by_gu)

※ 주거구별 정책 관심도(q2) 분포 [%]
q2     2     3     4     5     6    7
GU                                   
장안구  0.0  11.8  17.3  39.4  27.6  3.9
권선구  0.4   0.8  15.6  43.9  32.4  7.0
팔달구  2.6   6.4  44.9  20.5  21.8  3.8
영통구  2.3   6.9  23.7  40.5  20.6  6.1
※ 주거구별 정책 만족도(q3) 분포 [%]
q3     2    3     4     5     6    7
GU                                  
장안구  0.0  5.5  17.3  44.1  27.6  5.5
권선구  0.8  3.7   7.8  49.6  31.6  6.6
팔달구  0.0  2.6  26.9  35.9  32.1  2.6
영통구  1.5  3.8  22.9  35.9  32.8  3.1


주거구별 만족도, 불만족도 이유 파악

In [ ]:
# 1. 만족자만 필터링 (q3 >= 5)
satisfied = filtered_df[filtered_df['q3'] >= 5].copy()

# 2. 만족 이유 정리
satisfied_reason_by_gu = (
    satisfied
    .groupby('GU')['q3_1_1']  # q3_1_1: 만족 이유
    .value_counts(normalize=True)
    .mul(100)
    .round(1)
    .unstack(fill_value=0)
)

# 3. 주거구 이름 매핑 (선택)
gu_names = {1: '장안구', 2: '권선구', 3: '팔달구', 4: '영통구'}
satisfied_reason_by_gu.index = satisfied_reason_by_gu.index.map(gu_names)

# 4. 출력
print("✅ 주거구별 정책 만족 이유 분포 [%]")
print(satisfied_reason_by_gu)


✅ 주거구별 정책 만족 이유 분포 [%]
q3_1_1   1.0   2.0   3.0   4.0   5.0  6.0  7.0
GU                                            
장안구     17.2  29.0  26.9  11.8  10.8  4.3  0.0
권선구     13.2  50.7  19.5   8.8   4.4  2.9  0.5
팔달구     21.8  38.2  16.4  12.7   3.6  7.3  0.0
영통구     18.9  23.3  34.4  12.2   7.8  1.1  2.2


In [ ]:
# 1. 불만족자만 필터링 (q3 <= 3)
dissatisfied = filtered_df[filtered_df['q3'] <= 3].copy()

# 2. 불만족 이유 정리
dissatisfied_reason_by_gu = (
    dissatisfied
    .groupby('GU')['q3_1_2']  # q3_1_2: 불만족 이유
    .value_counts(normalize=True)
    .mul(100)
    .round(1)
    .unstack(fill_value=0)
)

# 3. 주거구 이름 매핑 (선택)
dissatisfied_reason_by_gu.index = dissatisfied_reason_by_gu.index.map(gu_names)

# 4. 출력
print("✅ 주거구별 정책 불만족 이유 분포 [%]")
print(dissatisfied_reason_by_gu)


✅ 주거구별 정책 불만족 이유 분포 [%]
Empty DataFrame
Columns: []
Index: []


주거구별/ 연령별 지역 생활 만족도

In [ ]:
# 1. 주거구별 q8_1 분포 계산 (비율)
q8_by_gu = (
    filtered_df
    .groupby('GU')['q8_1']
    .value_counts(normalize=True)
    .mul(100)
    .round(1)
    .unstack(fill_value=0)
)

# 2. 주거구 이름 매핑 (선택 사항)
gu_names = {1: '장안구', 2: '권선구', 3: '팔달구', 4: '영통구'}
q8_by_gu.index = q8_by_gu.index.map(gu_names)

# 3. 결과 출력
print("✅ 주거구별 지역 생활 만족도(q8_1) 분포 [%]")
print(q8_by_gu)


✅ 주거구별 지역 생활 만족도(q8_1) 분포 [%]
q8_1    1    2     3     4     5
GU                              
장안구   0.0  7.9  22.8  58.3  11.0
권선구   0.4  2.9  30.3  55.3  11.1
팔달구   0.0  2.6  14.1  80.8   2.6
영통구   0.8  5.3  19.8  59.5  14.5


In [ ]:
# 연령별 q8_1_1 분포 계산 (비율)
q8_by_age = (
    filtered_df
    .groupby('DM2_1')['q8_1']
    .value_counts(normalize=True)
    .mul(100)
    .round(1)
    .unstack(fill_value=0)
)

# 2. 연령 이름 매핑 (선택 사항)
age_labels = {
    1: '20대 이하', 2: '30대', 3: '40대', 4: '50대', 5: '60대 이상'
}
q8_by_age.index = q8_by_age.index.map(age_labels)

# 3. 결과 출력
print("✅ 연령별 지역 생활 만족도(q8_1) 분포 [%]")
print(q8_by_age)


✅ 연령별 지역 생활 만족도(q8_1) 분포 [%]
q8_1      1     2     3     4     5
DM2_1                              
20대 이하  0.0   3.6  23.8  61.9  10.7
30대     0.4   4.5  11.9  75.5   7.8
40대     0.5   4.8  40.3  41.9  12.4
50대     0.0   0.0  28.6  52.4  19.0
60대 이상  0.0  10.0  35.0  30.0  25.0


연령별 거주 기간 분포

In [ ]:
# 연령별 거주기간(q9) 분포 계산 (비율)
q9_by_age = (
    filtered_df
    .groupby('DM2_1')['q9']
    .value_counts(normalize=True)
    .mul(100)
    .round(1)
    .unstack(fill_value=0)
)

# 연령 라벨 매핑 (선택 사항)
age_labels = {
    1: '20대 이하', 2: '30대', 3: '40대', 4: '50대', 5: '60대 이상'
}
q9_by_age.index = q9_by_age.index.map(age_labels)

# 거주기간 라벨 매핑 (선택 사항)
q9_labels = {
    1: '1년 미만',
    2: '1~3년 미만',
    3: '3~5년 미만',
    4: '5~10년 미만',
    5: '10년 이상'
}
q9_by_age.columns = q9_by_age.columns.map(q9_labels)

# 결과 출력
print("✅ 연령별 수원 거주 기간(q9) 분포 [%]")
print(q9_by_age)


✅ 연령별 수원 거주 기간(q9) 분포 [%]
q9      1년 미만  1~3년 미만  3~5년 미만  5~10년 미만
DM2_1                                    
20대 이하    1.2     20.2     35.7      42.9
30대       0.4      5.2     18.6      75.8
40대       0.0      9.7     22.0      68.3
50대       9.5      4.8     19.0      66.7
60대 이상    5.0     10.0     25.0      60.0


연령별 중요하다고 생각하는 정

In [ ]:
# 연령별 q14 응답 분포 계산 (비율)
q14_by_age = (
    filtered_df
    .groupby('DM2_1')['q14']
    .value_counts(normalize=True)
    .mul(100)
    .round(1)
    .unstack(fill_value=0)
)

# 연령 라벨 (선택사항)
age_labels = {
    1: '20대 이하', 2: '30대', 3: '40대', 4: '50대', 5: '60대 이상'
}
q14_by_age.index = q14_by_age.index.map(age_labels)

# 결과 출력
print("✅ 연령별 q14 응답 분포 [%]")
print(q14_by_age)


✅ 연령별 q14 응답 분포 [%]
q14       1    2     3     4     5     6    7    8    9     10   11    12  \
DM2_1                                                                       
20대 이하   4.8  6.0  17.9  10.7   8.3  19.0  9.5  7.1  0.0   6.0  3.6   1.2   
30대      8.2  3.3  20.4   2.6  11.9  16.7  8.6  2.2  1.5  11.5  4.8   1.5   
40대      3.2  1.6  23.1   7.0   4.3  18.3  6.5  2.7  1.6  23.1  4.3   2.2   
50대     14.3  9.5  28.6   4.8   0.0   4.8  9.5  0.0  4.8   4.8  4.8  14.3   
60대 이상  10.0  0.0  65.0   5.0   0.0  10.0  5.0  0.0  5.0   0.0  0.0   0.0   

q14      13   14   15   16  
DM2_1                       
20대 이하  2.4  2.4  1.2  0.0  
30대     2.2  3.7  0.7  0.0  
40대     1.1  0.0  0.5  0.5  
50대     0.0  0.0  0.0  0.0  
60대 이상  0.0  0.0  0.0  0.0  
